In [0]:
!pip list

In [12]:
!pip install tensorboardX

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.datasets import CIFAR10
from Myutils import train, resnet
from torchvision import transforms as tfs
from datetime import datetime
from tensorboardX import SummaryWriter

In [14]:
def data_tf(x):
    im_aug = tfs.Compose([
        tfs.Resize(120),
        tfs.RandomHorizontalFlip(),
        tfs.RandomCrop(96),
        tfs.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
        tfs.ToTensor(),
        tfs.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    x = im_aug(x)
    return x

def test_tf(x):
    im_aug = tfs.Compose([
        tfs.Resize(96),
        tfs.ToTensor(),
        tfs.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    x = im_aug(x)
    return x

train_set = CIFAR10('./data', train=True, transform=data_tf,download=True)
test_set = CIFAR10('./data', train=False, transform=test_tf,download=True)

train_data = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, num_workers=4)
test_data = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False, num_workers=4)

net = resnet(3, 10)

optimizer = torch.optim.SGD(net.parameters(), lr=1e-1, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [15]:
writer = SummaryWriter()

def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().item()
    return num_correct / total

if torch.cuda.is_available():
    net = net.cuda()

prev_time = datetime.now()
for epoch in range(30):
    train_loss = 0
    train_acc = 0
    net = net.train()
    for im, label in train_data:
        if torch.cuda.is_available():
            im = Variable(im.cuda())
            label = Variable(label.cuda())
        else:
            im = Variable(im)
            label = Variable(label)
        output = net(im)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += get_acc(output, label)
    cur_time = datetime.now()
    h, remainder = divmod((cur_time - prev_time).seconds, 3600)
    m,s = divmod(remainder, 60)
    time_str = "Time %02d:%02d:%02d" % (h,m,s)
    
    valid_loss = 0
    valid_acc = 0
    net = net.eval()
    
    for im, label in test_data:
        if torch.cuda.is_available():
            im = Variable(im.cuda(), volatile=True)
            label = Variable(label.cuda(), volatile=True)
        else:
            im = Variable(im, volatile=True)
            label = Variable(label, volatile=True)
        output = net(im)
        loss = criterion(output, label)
        
        valid_loss += loss.item()
        valid_acc += get_acc(output, label)
        
    epoch_str = (
        "Epoch %d. Train Loss: %f, Train Acc; %f, Valid Loss: %f, Valid Acc:%f," %(epoch, train_loss/ len(train_data), train_acc/ len(train_data), valid_loss / len(test_data), valid_acc / len(test_data))
    )
    
    prev_time = cur_time
    
    writer.add_scalars('Loss', {'train': train_loss / len(train_data), 'valid':valid_loss / len(test_data)},epoch)
    writer.add_scalars('Acc', {'train': train_acc / len(train_data), 'valid':valid_acc / len(test_data)},epoch)
    
    print(epoch_str + time_str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Epoch 0. Train Loss: 1.853247, Train Acc; 0.322429, Valid Loss: 1.823401, Valid Acc:0.378711,Time 00:02:33
Epoch 1. Train Loss: 1.389418, Train Acc; 0.503894, Valid Loss: 2.502726, Valid Acc:0.344727,Time 00:02:41
Epoch 2. Train Loss: 1.133384, Train Acc; 0.600494, Valid Loss: 1.565982, Valid Acc:0.478125,Time 00:02:41
Epoch 3. Train Loss: 0.973005, Train Acc; 0.659546, Valid Loss: 1.570505, Valid Acc:0.498633,Time 00:02:40
Epoch 4. Train Loss: 0.865921, Train Acc; 0.698354, Valid Loss: 1.563231, Valid Acc:0.517480,Time 00:02:43
Epoch 5. Train Loss: 0.792180, Train Acc; 0.723585, Valid Loss: 1.581544, Valid Acc:0.535059,Time 00:02:39
Epoch 6. Train Loss: 0.734151, Train Acc; 0.744320, Valid Loss: 1.155796, Valid Acc:0.631641,Time 00:02:43
Epoch 7. Train Loss: 0.689023, Train Acc; 0.761464, Valid Loss: 1.257374, Valid Acc:0.616797,Time 00:02:43
Epoch 8. Train Loss: 0.648279, Train Acc; 0.774290, Valid Loss: 1.363479, Valid Acc:0.598145,Time 00:02:40
Epoch 9. Train Loss: 0.620541, Train 

In [16]:
!tensorboard --logdir runs

TensorBoard 1.12.2 at http://efd5e673a988:6006 (Press CTRL+C to quit)
^C


In [17]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2019-03-03 08:02:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.245.79, 52.73.9.93, 52.71.139.107, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.245.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  20.2MB/s    in 0.3s    

2019-03-03 08:02:39 (20.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]



In [18]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './runs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [21]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://a76c3ff9.ngrok.io
